# Experiment

## Install Python dependencies

In [ ]:
!pip install onnx onnxruntime tf2onnx

Import the dependencies for the model training code:

In [ ]:
import numpy as np
import pandas as pd
import datetime
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
import tf2onnx
import onnx
import pickle
from pathlib import Path

The output might show TensorFlow messages, such as a "Could not find TensorRT" warning. You can ignore these messages.


## Load the CSV data

The CSV data that you use to train the model contains the following fields:
*  **Call_Duration**
*  **Data_Usage**
*  **SMS_Count**
*  **Roaming_Indicator**
*  **MobileWallet_Use**
*  **Cost**
*  **Cellular_Location_Distance**
*  **Personal_Pin_Used** 
*  **Avg_Call_Duration**
*  **Avg_Data_Usage**
*  **Avg_Cost**
*  **fraud** - If the transaction is fraudulent.

In [ ]:
# Set the input (X) and output (Y) data. 
# The only output data is whether it's fraudulent. All other fields are inputs to the model.

feature_indexes = [
    0,  # Call_Duration
    1,  # Data_Usage
    2,  # SMS_Count
    3,  # Roaming_Indicator
    4,  # MobileWallet_Use
    6,  # Cost
    7,  # Cellular_Location_Distance
    8,  # Personal_Pin_Used 
    9,  # Avg_Call_Duration
    10, # Avg_Data_Usage
    11  # Avg_Cost
]

label_indexes = [
    12  # fraud
]

df = pd.read_csv('data/telecom_revass_data.csv')
X = df.iloc[:, feature_indexes].values
y = df.iloc[:, label_indexes].values

print(df.info)

print(df.head)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

Path("artifact").mkdir(parents=True, exist_ok=True)
with open("artifact/test_data.pkl", "wb") as handle:
    pickle.dump((X_test, y_test), handle)
with open("artifact/scaler.pkl", "wb") as handle:
    pickle.dump(scaler, handle)

# Since the dataset is unbalanced (it has many more non-fraud transactions than fraudulent ones), set a class weight to weight the few fraudulent transactions higher than the many non-fraud transactions.
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train.ravel())
class_weights = {i : class_weights[i] for i in range(len(class_weights))}

## Build the model

The model is a simple, fully-connected, deep neural network, containing three hidden layers and one output layer.

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=len(feature_indexes)))
model.add(Dropout(0.2))
model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

## Train the model

Training a model is often the most time-consuming part of the machine learning process.  Large models can take multiple GPUs for days.  Expect the training on CPU for this very simple model to take a minute or more.

In [ ]:
# Train the model and get performance
import os
import time

start = time.time()
epochs = 2
history = model.fit(
    X_train,
    y_train,
    epochs=epochs,
    verbose=True,
    class_weight=class_weights
)
end = time.time()
print(f"Training of model is complete. Took {end-start} seconds")

## Save the model file

In [ ]:
import tensorflow as tf

# Normally we use tf2.onnx.convert.from_keras.
# workaround for tf2onnx bug https://github.com/onnx/tensorflow-onnx/issues/2348

# Wrap the model in a `tf.function`
@tf.function(input_signature=[tf.TensorSpec([None, X_train.shape[1]], tf.float32, name='dense_input')])
def model_fn(x):
    return model(x)

# Convert the Keras model to ONNX
model_proto, _ = tf2onnx.convert.from_function(
    model_fn,
    input_signature=[tf.TensorSpec([None, X_train.shape[1]], tf.float32, name='dense_input')]
)

# Save the model as ONNX for easy use of ModelMesh
os.makedirs("models/fraud/1", exist_ok=True)
onnx.save(model_proto, "models/fraud/1/rafmmodel.onnx")

The output might include TensorFlow messages related to GPUs. You can ignore these messages.

## Confirm the model file was created successfully

The output should include the model name, size, and date. 

In [ ]:
! ls -alRh ./models/

## Test the model

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import pickle
import onnxruntime as rt

Load the test data and scaler:

In [ ]:
with open('artifact/scaler.pkl', 'rb') as handle:
    scaler = pickle.load(handle)
with open('artifact/test_data.pkl', 'rb') as handle:
    (X_test, y_test) = pickle.load(handle)

Create an ONNX inference runtime session and predict values for all test inputs:

In [ ]:
sess = rt.InferenceSession("models/fraud/1/rafmmodel.onnx", providers=rt.get_available_providers())
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name
y_pred_temp = sess.run([output_name], {input_name: X_test.astype(np.float32)}) 
y_pred_temp = np.asarray(np.squeeze(y_pred_temp[0]))
threshold = 0.95
y_pred = np.where(y_pred_temp > threshold, 1, 0)

Show the results:

In [ ]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
import numpy as np

y_test_arr = y_test.squeeze()
correct = np.equal(y_pred, y_test_arr).sum().item()
acc = (correct / len(y_pred)) * 100
precision = precision_score(y_test_arr, np.round(y_pred))
recall = recall_score(y_test_arr, np.round(y_pred))

print(f"Eval Metrics: \n Accuracy: {acc:>0.1f}%, "
      f"Precision: {precision:.4f}, Recall: {recall:.4f} \n")

c_matrix = confusion_matrix(y_test_arr, y_pred)
ConfusionMatrixDisplay(c_matrix).plot()

## Example: Is below transaction likely to be fraudulent?

Here is the order of the fields from Sally's transaction details:
 Call_Duration 10
 
 Data_Usage 300
 
 SMS_Count 5
 
 Roaming_Indicator 0
 
 MobileWallet_Use 1
 
 Cost 50
 
 Cellular_Location_Distance 3
 
 Personal_Pin_Used  20
 
 Avg_Call_Duration 12
 
 Avg_Data_Usage 350
 
 Avg_Cost 0

In [ ]:

# No-fraud
# Call_Duration 10
# Data_Usage 300
# SMS_Count 5
# Roaming_Indicator 0
# MobileWallet_Use 1
# Cost 50
# Cellular_Location_Distance 3
# Personal_Pin_Used  20
# Avg_Call_Duration 12
# Avg_Data_Usage 350
# Avg_Cost


telco_transaction_details = [
    [10,
    300,
    5,
    0,
    1,
    50,
    3,
    20,
    12,
    350,
     0
     ]
    ]
prediction = sess.run([output_name], {input_name: scaler.transform(telco_transaction_details).astype(np.float32)})

print("Is transaction predicted to be fraudulent? (true = YES, false = NO) ")
print(np.squeeze(prediction) > threshold)

print("How likely was this transaction to be fraudulent? ")
print("{:.5f}".format(100 * np.squeeze(prediction)) + "%")

## Example: Is below transaction likely to be fraudulent?

Here is the order of the fields from Sally's transaction details:
 #Fraud#
 
 Call_Duration 300
 
 Data_Usage 10000
 
 SMS_Count 50
 
 Roaming_Indicator 1
 
 MobileWallet_Use 1
 
 Cost 500
 
 Cellular_Location_Distance 100
 
 Personal_Pin_Used  1
 
 Avg_Call_Duration 50
 
 Avg_Data_Usage 8000
 
 Avg_Cost 0

In [ ]:
telco_transaction_details = [
    [300,
    10000,
    50,
    1,
    1,
    500,
    100,
    1,
    50,
    8000,
     0
          ]
    ]

prediction = sess.run([output_name], {input_name: scaler.transform(telco_transaction_details).astype(np.float32)})

print("Is transaction predicted to be fraudulent? (true = YES, false = NO) ")
print(np.squeeze(prediction) > threshold)

print("How likely was this transaction to be fraudulent? ")
print("{:.5f}".format(100 * np.squeeze(prediction)) + "%")